# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## NAME: KAILASH BALACHANDIRAN
## STUDENT ID: 220243160

## Coursework overview

                                                                                                                                                       

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips datase:
    df = trips.filter((trips.trip_distance != 0) | ((trips.passenger_count != 0) & (trips.total_amount != 0)))
    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 2,890,208


In [0]:
# Your solution implementation to task 1.2 goes HERE
from pyspark.sql.window import Window
from pyspark.sql.functions import abs
import sys
import numpy as np
def t12_remove_outliers(df):
    #total_amount
    median = df.agg(pf.percentile_approx(df.trip_distance, 0.5, 10000)).alias("median").collect()[0][0]
    subTrip = abs(df.trip_distance - median)
    df = df.withColumn("subtractedTrip", pf.lit(subTrip))
    madTrip = 1.438 * (df.agg(pf.percentile_approx(df.subtractedTrip, 0.5, 10000)).alias("median").collect()[0][0])
    df = df.withColumn("madTrip", pf.lit(madTrip))
    zScoreAmount = abs(subTrip/madTrip)
    df = df.withColumn("zscoreTrip", pf.lit(zScoreAmount))
    
    # total_distance
    totalMedian = df.agg(pf.percentile_approx(df.total_amount, 0.5,1000)).alias("median").collect()[0][0]
    subTotal = abs(df.total_amount - median)
    df = df.withColumn("subtractedAmount", pf.lit(subTotal))
    madTotal = 1.438 * (df.agg(pf.percentile_approx(df.subtractedAmount, 0.5, 10000)).alias("median").collect()[0][0])
    df = df.withColumn("madAmount", pf.lit(madTotal))
    zScoreDist = abs(subTotal/madTotal)
    df = df.withColumn("zscoreAmount", pf.lit(zScoreDist))
    
    df = df.filter((df.zscoreTrip < 3.5) & (df.zscoreAmount < 3.5))
    
    df = df.drop("subtractedTrip","madTrip","zscoreTrip","subtractedAmount","madAmount","ascoreAmount")
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 2,538,711


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 2.1 goes HERE
from pyspark.sql.functions import col
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    df = df.join(zones_df, df.PULocationID == zones_df.LocationID).select(df["*"],zones_df["Zone"])
    df = df.withColumnRenamed("Zone","start")
    
    zones_df = zones_df.select("LocationID",col("Zone").alias("New_Zone"))
    df = df.join(zones_df, df.DOLocationID == zones_df.LocationID).select(df["*"],zones_df["New_Zone"])
    df = df.withColumnRenamed("New_Zone","end")
    
    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 2,538,711


In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    df = df.withColumn("unit_profitability", pf.lit(abs(df.total_amount/df.trip_distance)))
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
#display(trips_22.take(10))
display(trips_22)

Row count: 2,538,711


index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,cab_type,lpep_pickup_datetime,lpep_dropoff_datetime,ehail_fee,trip_type,zscoreAmount,start,end,unit_profitability
0,1.0,2021-07-01 00:08:51,2021-07-01 00:13:05,1.0,0.8,1.0,N,90,68,1.0,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,Y,null,null,null,null,0.358084404126793,Flatiron,East Chelsea,11.0
1,1.0,2021-07-01 00:22:39,2021-07-01 00:25:58,1.0,0.9,1.0,N,113,90,2.0,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,Y,null,null,null,null,0.358084404126793,Greenwich Village North,Flatiron,9.777777777777779
2,1.0,2021-07-01 00:48:33,2021-07-01 00:54:58,1.0,2.8,1.0,N,88,232,2.0,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,Y,null,null,null,null,0.617565856392585,Financial District South,Two Bridges/Seward Park,4.928571428571429
3,1.0,2021-07-01 00:59:44,2021-07-01 01:07:09,1.0,1.4,1.0,N,79,249,1.0,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,Y,null,null,null,null,0.5397214207128475,East Village,West Village,8.785714285714286
4,1.0,2021-07-01 00:08:35,2021-07-01 00:16:28,0.0,2.0,1.0,N,142,238,1.0,8.5,3.0,0.5,0.0,0.0,0.3,12.3,2.5,Y,null,null,null,null,0.5397214207128475,Lincoln Square East,Upper West Side North,6.15
5,1.0,2021-07-01 00:10:49,2021-07-01 00:18:42,1.0,1.6,1.0,N,114,90,1.0,7.5,3.0,0.5,1.5,0.0,0.3,12.8,2.5,Y,null,null,null,null,0.5656695659394266,Greenwich Village South,Flatiron,8.0
6,1.0,2021-07-01 00:28:56,2021-07-01 00:36:11,1.0,1.8,1.0,N,90,144,1.0,7.5,3.0,0.5,2.0,0.0,0.3,13.3,2.5,Y,null,null,null,null,0.5916177111660058,Flatiron,Little Italy/NoLiTa,7.388888888888889
7,1.0,2021-07-01 00:45:10,2021-07-01 00:52:46,1.0,2.0,1.0,N,114,48,1.0,8.5,3.0,0.5,2.45,0.0,0.3,14.75,2.5,Y,null,null,null,null,0.6668673323230855,Greenwich Village South,Clinton East,7.375
8,1.0,2021-07-01 00:55:43,2021-07-01 01:10:56,1.0,5.7,1.0,N,48,152,2.0,18.5,3.0,0.5,0.0,0.0,0.3,22.3,2.5,Y,null,null,null,null,1.0586843252444316,Clinton East,Manhattanville,3.912280701754386
9,1.0,2021-07-01 00:04:24,2021-07-01 00:13:23,1.0,1.8,1.0,N,234,148,1.0,8.5,3.0,0.5,2.45,0.0,0.3,14.75,2.5,Y,null,null,null,null,0.6668673323230855,Union Sq,Lower East Side,8.194444444444445


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
## Your solution to task 3.1 goes HERE
from pyspark.sql.functions import avg, mean, sum
import pyspark.sql.utils
def t31_summarise_trips(df):
    # input: output of task 2.2
    df = df.groupby(df.PULocationID, df.DOLocationID).agg(pf.mean(df.unit_profitability),pf.count(df.PULocationID),pf.sum(df.passenger_count))
    df = df.withColumnRenamed('avg(unit_profitability)','average_unit_profitability').withColumnRenamed('count(PULocationID)', 'trips_count').withColumnRenamed('sum(passenger_count)', 'total_passengers')
    return df




In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 17,454


PULocationID,DOLocationID,average_unit_profitability,trips_count,total_passengers
90,231,7.469893120559336,1126,1579.0
87,33,6.817160729833857,105,145.0
90,142,6.872020505232908,910,1280.0
114,100,7.480600705110231,466,707.0
234,144,9.132186157728823,1489,2188.0
148,262,4.852589259878061,279,413.0
246,249,8.817535758115456,1151,1714.0
170,179,5.138416672570028,80,103.0
48,232,5.436803495465059,358,519.0
142,144,5.79057383648395,353,494.0


In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    df = df.groupby(df.PULocationID).agg(pf.sum(df.trips_count), pf.mean(df.average_unit_profitability), pf.sum(df.total_passengers))
    df = df.withColumnRenamed('sum(trips_count)', 'total_trips').withColumnRenamed('avg(average_unit_profitability)', 'average profitability').withColumnRenamed('sum(total_passengers)', 'total passenger').join(zones_df, df.PULocationID == zones_df.LocationID, how = 'left')
    df = df.drop('Borough').drop('service_zone')
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.orderBy(col('total_trips').desc()).take(10)
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.orderBy(col('average profitability').desc()).take(10)
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.orderBy(col('total passenger').desc()).take(10)
    return df_zones

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

PULocationID,total_trips,average profitability,total passenger,LocationID,Zone
237,123407,7.646555928677428,172033.0,237,Upper East Side South
236,105072,6.979484429256904,145730.0,236,Upper East Side North
186,98289,10.930086418041705,139346.0,186,Penn Station/Madison Sq West
161,94964,8.430137067819656,136785.0,161,Midtown Center
170,89789,9.991756440443568,125613.0,170,Murray Hill
162,86387,7.9017548520981915,121418.0,162,Midtown East
142,82045,7.804087577291357,116186.0,142,Lincoln Square East
48,75921,8.87966179291784,109070.0,48,Clinton East
239,75829,8.154611278249472,107375.0,239,Upper West Side South
141,75381,11.119831042584911,105110.0,141,Lenox Hill West


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

PULocationID,total_trips,average profitability,total passenger,LocationID,Zone
1,46,300.23702571854193,55.0,1,Newark Airport
15,74,288.39417611738327,35.0,15,Bay Terrace/Fort Totten
206,6,147.6813775856173,3.0,206,Saint George/New Brighton
120,26,122.67460688200988,33.0,120,Highbridge Park
227,169,119.85519047814084,30.0,227,Sunset Park East
70,1042,104.44136529397991,1433.0,70,East Elmhurst
14,364,98.6774248663507,151.0,14,Bay Ridge
93,72,97.08803207215891,80.0,93,Flushing Meadows-Corona Park
190,79,86.20174291896654,71.0,190,Prospect Park
118,7,80.72293283135613,4.0,118,Heartland Village/Todt Hill


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

PULocationID,total_trips,average profitability,total passenger,LocationID,Zone
237,123407,7.646555928677428,172033.0,237,Upper East Side South
236,105072,6.979484429256904,145730.0,236,Upper East Side North
186,98289,10.930086418041705,139346.0,186,Penn Station/Madison Sq West
161,94964,8.430137067819656,136785.0,161,Midtown Center
170,89789,9.991756440443568,125613.0,170,Murray Hill
162,86387,7.9017548520981915,121418.0,162,Midtown East
142,82045,7.804087577291357,116186.0,142,Lincoln Square East
48,75921,8.87966179291784,109070.0,48,Clinton East
239,75829,8.154611278249472,107375.0,239,Upper West Side South
79,74930,9.260490093904522,107156.0,79,East Village


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: XXL
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-XXL-delta/
      Number of trips (dataset rows): 132,396,785
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[130]: [[Row(PULocationID=237, total_trips=5565676, average profitability=5.901268269659499, total passenger=8289121.0, LocationID=237, Zone='Upper East Side South'),
  Row(PULocationID=236, total_trips=5134220, average profitability=5.392178189255118, total passenger=7648845.0, LocationID=236, Zone='Upper East Side North'),
  Row(PULocationID=161, total_trips=4896482, average profitability=7.013543023316442, total passenger=7486208.0, LocationID=161, Zone='Midtown Center'),
  Row(PULocationID=186, total_trips=4386952, average profitability=6.567074316723475, total passenger=6551102.0, LocationID=186, Zone='Penn Station/Madison Sq West'),
  Row(PULocationID=162, total_trips=4326332, average profitability=6.723867941367279, total passenger=6498591.0, LocationID=162, Zone='Midtown East'),
  Row(PULocationID=230, total_trips=3868179, average profitability=7.488279066143349, total passenger=6135176.0, LocationID=230, Zone='Times Sq/Theatre District'),
  Row(PULocationID=170, total_trips

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.89 |  15.57 |  41.95 |  90.44 |  132.39 |
| execution time   (w/o 1.2)  | 11.08 sec  | 9.03 sec | 817.2 sec (13.62 min) | 1675.2 sec (27.92 min) | 1212 sec (20.20 min) |
| execution time              | 17.72 sec | 22.08 sec | 1785.6 sec (29.76 min) | 2430.6 sec (40.51 min) | 2719.2 sec (45.32 min) |
| sec / 1M records (w/o 1.2)  | 3.833  | 0.579  | 19.48  | 18.52  | 9.154  |
| sec / 1M records            | 6.131  | 1.418  | 42.56  | 26.87  | 20.53  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.89 |  15.57 |  41.95 |  90.44 |  132.39 |
| execution time   (w/o 1.2)  | 10.37 sec | 10.60 sec | 10.26 sec | 10.95 sec | 12.65 sec |
| execution time              | 13.45 sec | 19.18 sec | 20.40 sec | 25.71 sec | 28.35 sec |
| sec / 1M records (w/o 1.2)  | 3.588  | 0.680  | 0.244  | 0.121  | 0.09  |
| sec / 1M records            | 4.653  | 1.231  | 0.486  | 0.284  | 0.214  |